# Titanic Survival Prediction Project
This notebook uses the Titanic dataset to predict whether a passenger survived or not.

In [1]:
# 📦 Step 1: Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [6]:
# 📤 Step 2: Upload CSV files (if using Colab)
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive (1).zip


In [27]:
import zipfile
import os

# Extract the contents of the uploaded zip file
with zipfile.ZipFile("archive.zip", 'r') as zip_ref:
    zip_ref.extractall()

# List all extracted files and folders
print(os.listdir())


['.config', 'submission.csv', 'archive.zip', 'archive (1).zip', 'archive (2).zip', 'Titanic-Dataset.csv', 'sample_data']


In [9]:
# Load the dataset
data = pd.read_csv('Titanic-Dataset.csv')

# Display the first few rows
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)


In [14]:
# 🔍 Step 4: Explore Data
print(train.info())
print(train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 331 to 102
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          572 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Cabin        159 non-null    object 
 11  Embarked     710 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 72.3+ KB
None
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            140
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          553
Embarked         2
dtype: int64


In [15]:
# 🧹 Step 5: Clean Data
train['Age'].fillna(train['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)
train.drop('Cabin', axis=1, inplace=True)
test.drop('Cabin', axis=1, inplace=True)

<ipython-input-15-25746067>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['Age'].fillna(train['Age'].median(), inplace=True)
<ipython-input-15-25746067>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [16]:
# 🏗️ Step 6: Feature Engineering
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
for dataset in [train, test]:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt','Col',
     'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

label = LabelEncoder()
for col in ['Sex', 'Embarked', 'Title']:
    train[col] = label.fit_transform(train[col])
    test[col] = label.transform(test[col])

train.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
test_passenger_ids = test['PassengerId']
test.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

In [21]:
# 📊 Step 7: Split Data
X = train.drop('Survived', axis=1)
y = train['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# 🤖 Step 8: Train Models
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)
log_preds = log_model.predict(X_val)
print('Logistic Regression Accuracy:', accuracy_score(y_val, log_preds))

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_val)
print('Random Forest Accuracy:', accuracy_score(y_val, rf_preds))
print('Random Forest Report:', classification_report(y_val, rf_preds))

Logistic Regression Accuracy: 0.8041958041958042
Random Forest Accuracy: 0.8321678321678322
Random Forest Report:               precision    recall  f1-score   support

           0       0.85      0.87      0.86        87
           1       0.80      0.77      0.78        56

    accuracy                           0.83       143
   macro avg       0.83      0.82      0.82       143
weighted avg       0.83      0.83      0.83       143



In [28]:
# 📈 Step 9: Predict on Test Set
# Drop the 'Survived' column from the test set if it exists
if 'Survived' in test.columns:
    test_features = test.drop(columns=['Survived'])
else:
    test_features = test

# Make predictions
test_predictions = rf_model.predict(test_features)

# Create submission DataFrame
submission = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': test_predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

# Download it in Colab
from google.colab import files
files.download('submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>